In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import sys
sys.path.append("..")
from body_parts_measurement_data_generator import BodyPartsMeasurementDataGenerator

In [2]:
config = {
    # model
    'input_shape': [256,256,3],     # 훈련 이미지 크기
    'batch_size': 4,                # 배치 사이즈
    'path_pretrained': None,        # pretrained 모델 경로
    'type_backbone': "blazepose",   # backbone type (blazepose, mobilenet_v3)
    
    # loss
    'type_loss_fn': 'wing',         # 손실 함수 설정 (wing, mae)
    
    # data
    'seg_shape': [64,64],           # segmentation 크기 *미사용
    'path_classes': "../seg_classes.txt",   # segmentation class 정보 *미사용
    'shuffle': True,                # 데이터 섞기
    'is_normalized': False,         # normalize 데이터
    'is_with_seg': False,           # segmentation 사용 여부 *미사용
    'path_dataset': "D:\\data\\body_parts_measurement", # 데이터 경로
    ## attention type              
    'type_attention': "regression", # attention 종류 (regression, categorical, none)
    'num_category_bmi': 10,         # categorical 시의 bmi category 갯수 변수
    'num_category_height': 10,      # categorical 시의 height category 갯수 변수

    # train
    'epochs': 30,                   # 훈련 epoch 수
    'eval_term': 1                  # 평가 빈도
}

In [3]:
data_generator = BodyPartsMeasurementDataGenerator(config, data_type="test")

list_user_dirs ['F371', 'F372', 'F373', 'F374', 'F375', 'F376', 'M401', 'M402', 'M403', 'M404', 'M405']


In [8]:
print(data_generator.min_body_parts_measurement, data_generator.min_body_parts_measurement.shape)
print(data_generator.max_body_parts_measurement, data_generator.max_body_parts_measurement.shape)

[122.4  72.9 107.   90.5  64.   38.   54.5  31.   82.5  68.6  75.5  90.8
  49.8  33.8  32.7  20.   23.5  26.5  22.   14.5  27.8  49.1  34.1  20.1
   9.7  20.9   8.8  13.3  16.    9.    7.2] (31,)
[147.8  85.9 131.1 107.9  78.1  45.6  58.4  42.  106.3  91.2  96.5 102.
  60.6  38.5  41.2  23.2  27.9  36.4  28.1  18.2  33.5  57.7  41.9  23.2
  11.9  25.6  10.9  15.9  18.9  11.3   9.2] (31,)


In [5]:
for idx in range(10) :
    data = data_generator.__getitem__(idx)
    print(data[1])

[[0.7362206  0.73846143 0.5518672  0.5919542  0.609929   0.5
  0.58974314 0.36363637 0.60924363 0.65929216 0.5619049  0.68749994
  0.7407408  0.89361703 0.48235276 0.37500015 0.11363637 0.5858584
  0.57377046 0.54054046 0.50877213 0.5697675  0.5128203  0.41935453
  0.31818175 0.57446814 0.2380953  0.4230768  0.8275862  0.521739
  0.4000001 ]
 [1.         1.         1.         1.         1.         1.
  0.7435898  0.59999985 0.5378152  0.579646   0.52380955 0.41071427
  0.5555556  0.44680893 0.6705883  0.12499985 0.4545455  0.5050504
  0.40983605 0.48648617 1.         1.         0.8076924  0.3225806
  0.3636365  1.         0.42857137 0.5769232  0.37931052 0.39130414
  0.5500002 ]
 [0.         0.04615373 0.         0.         0.         0.05263179
  0.25641015 0.27272728 0.6680672  0.8761064  1.         1.
  0.8055557  0.12766004 0.39999974 0.34375003 0.         0.5050504
  0.42622954 0.54054046 0.         0.         0.05128223 0.58064485
  0.04545472 0.40425518 0.33333334 0.30769244 0.2

In [4]:
data = data_generator.__getitem__(2)

In [ ]:
img = data[0].astype(np.int32)

In [ ]:
np.amax(data[0])

In [ ]:
data[1].shape

In [ ]:
data[1]

In [ ]:
print(np.amax(data[1]))

In [ ]:
data[2][0,:,:,0].shape

In [ ]:
plt.imshow(data[0][0,:,:,:])
plt.show()

plt.imshow(img[0,:,:,:])
plt.show()

plt.imshow(data[2][0,:,:,0])
plt.show()

In [ ]:
np.amax(data[2][:,:,3])

In [ ]:
cv2.resize(data, (128,128))

In [ ]:
iter_dg = iter(data_generator)

In [ ]:
data = next(iter_dg)

In [ ]:
data[0].shape

In [ ]:
data[1].shape

In [ ]:
data[1] / data_generator.max_body_parts_measurement

In [ ]:
path_base_dataset_dir = "C:\\Users\\user\\work\\data\\body_parts_measurement"
list_user_dirs = os.listdir(path_base_dataset_dir)
list_user_dirs.sort()
print(len(list_user_dirs))

list_data = []
list_body_parts_measurement = []
list_max_body_parts_measurement = [0] * 36
for user_dir in list_user_dirs :
    path_full_user_dir = path_base_dataset_dir + os.path.sep + user_dir
    ##### get parts measurement
    path_user_csv = path_full_user_dir + os.path.sep + "csv" + os.path.sep + user_dir + ".csv"
    df = pd.read_csv(path_user_csv, encoding = "ISO-8859-1")
    body_parts_measurement = df.iloc[[1]].values[0][3:-1]
    # change gender to digit
    if body_parts_measurement[-2] == "F" : 
        body_parts_measurement[-2] = "1"
    elif body_parts_measurement[-2] == "M" :
        body_parts_measurement[-2] = "0"
    else :
        print("wrong")
    body_parts_measurement = body_parts_measurement.astype(np.float32)
    # get max value for each column for normalization
    for idx, (max_val, measurement) in enumerate(zip(list_max_body_parts_measurement, body_parts_measurement)) :
        if measurement > max_val :
            list_max_body_parts_measurement[idx] = measurement
    ##### get list of images
    path_user_image_dir = path_full_user_dir + os.path.sep + "image"
    # get only images for attention posture
    list_path_images = glob.glob(path_user_image_dir + os.path.sep + "*.jpg")
    for each_path_image in list_path_images :
        each_data = [each_path_image, body_parts_measurement]
        list_data.append(each_data)


In [ ]:
list_max_body_parts_measurement

In [ ]:
import labelme
import imgviz
import json
import codecs
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [ ]:
print("한글")

In [ ]:
file_name = "C:\\Users\\user\\work\\data\\body_parts_measurement\\train\\F001\\json\\01_01_F001_01.json"

In [ ]:
codec_f = codecs.open(file_name, 'r', encoding='euc_kr') 

In [ ]:
data = json.load(codec_f)

In [ ]:
data['labelingInfo'][0]

In [ ]:
coordinates = data['labelingInfo'][0]['polygon']['location'].split(" ")[:-1]

In [ ]:
list_coords = [int(x) for x in coordinates]
contours = np.array(list_coords).reshape(-1,2)

In [ ]:
image = np.zeros((3000,3000))
cv2.fillPoly(image, pts=[contours], color=(255,0,0))
plt.imshow(image)
plt.show()

In [ ]:
data

In [ ]:
import site; print(''.join(site.getsitepackages()))